In [1]:
%matplotlib widget
import numpy as np

from structure import hydrostat_arm_3d
from render import draw_nodes_3d
from structure import polytopes
from structure import structures
from environment import environment
from environment import obstacle

import importlib

pygame 2.6.1 (SDL 2.28.4, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


# 3D Hydrostat Simulation

The framework for this simulation matches that of the 2D simulation. In order to generalize the hydrostat model for cells that are not simplices (not triangles, tetrahedrons, etc) and for 3D polytopes, it is necessary to define general volume constraints, facet constraints (planar points must remain planar), and self intersection constraints (vertices going through arm edges). 

## General volume calculation
There are three proposed methods of calculating the volume constraint.
1. Use the Qhull methods via scipy wrapper. Pros: out of the box way to calculate volume. Cons: unable to update point coordinates, so new objects must be created for each cell at each time step. Probably slow. Also, not easily differentiable for calculating the Jacobian.
2. Triangulate the shape into simplices, calculate each individual tetrahedron, and then sum the volumes. Pros: Triangulation can be done once at initialization and the point sets stored for iterating. Caclulating the volume of tetrahedrons is simple. Number of calculations scales linearly with the number of vertices. Cons: Need to figure out how to break into simplices.
3. Break the solid into pyramids, calculate the volume of each pyramid, and then sum the volumes. Pros: Point sets can be determined at initialization. Calculations scale a little less than linearly with the number of vertices since only the area calculations would scale with vertices. Volume calculations would scale with facets. Cons: Not entirely sure how the differentiating the dot product necessary in this calculation would work. 

## Facet Constraints
Certain points would need to stay on the same facet. In our 3D case this means the points on a face must stay on some shared 2D plane. This can be done by specifying that the relative vectors to a single point are all orthogonal to the face's normal vector.

## Self Intersection
In order to prevent vertices from going through edges, we can add half-space constraints for each vertex and edge pair. This is rather coputationally expensive but maybe it's okay? Grows at V x E.

In [2]:

importlib.reload(hydrostat_arm_3d)
importlib.reload(polytopes)
importlib.reload(obstacle)
importlib.reload(environment)

z = 6
cube_arm = polytopes.CubeArm(height = z)
# points = polytopes.Cube.points
# vertices = polytopes.Cube.vertices
# edges = polytopes.Cube.edges
# faces = polytopes.Cube.faces

cells = []
for idx, cell in enumerate(cube_arm.cells):
    masses = np.ones_like(cell.vertices) / len(cell.vertices)
    # vertex_damping = np.ones_like(cell.vertices) * 0.1
    vertex_damping = np.ones_like(cell.vertices) / len(cell.vertices)
    if idx == 0:
        cells.append(hydrostat_arm_3d.HydrostatCell3D(cell.vertices, cell.edges,
                                                      cell.faces, fixed_indices=[0, 1, 2, 3], 
                                                      masses=masses, vertex_damping=vertex_damping))
    else:
        cells.append(hydrostat_arm_3d.HydrostatCell3D(cell.vertices, cell.edges, cell.faces, masses=masses, vertex_damping=vertex_damping))
arm = hydrostat_arm_3d.HydrostatArm3D(cube_arm.points / np.array([1,1,1]), cells)
# _ = arm.set_external_forces(1, [0, 0, -1])
# arm.set_muscle_actuations([4,5,6,7], 10)


In [2]:
importlib.reload(structures)
importlib.reload(polytopes)
importlib.reload(obstacle)
importlib.reload(environment)

z = 6
cube_arm = polytopes.CubeArm(height = z)

cells = []
for idx, cell in enumerate(cube_arm.cells):
    masses = np.ones_like(cell.vertices) / len(cell.vertices)
    # vertex_damping = np.ones_like(cell.vertices) * 0.1
    vertex_damping = np.ones_like(cell.vertices) / len(cell.vertices)
    if idx == 0:
        cells.append(structures.Cell3D(cell.vertices, cell.edges,
                                                      cell.faces, fixed_indices=[0, 1, 2, 3], 
                                                      masses=masses, vertex_damping=vertex_damping))
    else:
        cells.append(structures.Cell3D(cell.vertices, cell.edges, cell.faces, masses=masses, vertex_damping=vertex_damping))
arm = structures.Arm3D(cube_arm.points, np.zeros_like(cube_arm.points), cells, None, None)
# _ = arm.set_external_forces(1, [0, 0, -1])
# arm.set_muscle_actuations([4,5,6,7], 10)

In [3]:
obst = obstacle.ConvexObstacle3D(
    vertices = (polytopes.Cube.points * np.array([1, 10, 10]) + np.array([-2.5, -1, -1])),
    # vertices = polytopes.Cube.points,
    edges = polytopes.Cube.edges,
    faces = polytopes.Cube.faces
)

food_array = np.array([[-4,0,3, 1]])
# food_array = np.array([[-2, 0, 0, 1]])
env = environment.Environment3D([obst], food_array, dx=.2)
arm.set_environment(env)


Loop: 50, Error: 1.000
Loop: 100, Error: 1.000
Loop: 150, Error: 0.372
Loop: 200, Error: 0.186
Loop: 250, Error: 0.114
Loop: 300, Error: 0.077
Loop: 350, Error: 0.055


In [4]:
# import pickle

# # # with open("saved_environment.pkl", "wb") as f:
# # #     pickle.dump(env, f)
# with open("saved_environment.pkl", "rb") as f:
#     env = pickle.load(f)
#     arm.set_environment(env)

In [5]:
importlib.reload(draw_nodes_3d)
drawer = draw_nodes_3d.NodeDrawer3D(arm, dt=1/60)
drawer.main_loop(simulating = False)

Actual FPS: 0.00 | Loop Period (ms): 16.30 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 15.99 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 16.68 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 16.96 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 16.85 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 15.96 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 16.10 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 15.80 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 16.32 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 15.89 | Sim Time (ms): 0.00
Actual FPS: 61.35 | Loop Period (ms): 15.91 | Sim Time (ms): 0.00
Actual FPS: 61.35 | Loop Period (ms): 15.93 | Sim Time (ms): 0.00
Actual FPS: 61.35 | Loop Period (ms): 16.14 | Sim Time (ms): 0.00
Actual FPS: 61.35 | Loop Period (ms): 16.32 | Sim Time (ms): 0.00
Actual FPS: 61.35 | Loop Period (ms): 16.12 | Sim Time (ms): 0.00
Actual FPS: 61.35 | 

SystemExit: 

/home/alex/code/muscular_hydrostat/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
importlib.reload(draw_nodes_3d)
drawer = draw_nodes_3d.NodeDrawer3D(arm, dt=1/60)
drawer.main_loop(simulating = True, rotating=True, recording=True)

Actual FPS: 0.00 | Loop Period (ms): 78.85 | Sim Time (ms): 38.89
Actual FPS: 0.00 | Loop Period (ms): 15.06 | Sim Time (ms): 12.02
Actual FPS: 0.00 | Loop Period (ms): 27.30 | Sim Time (ms): 21.57
Actual FPS: 0.00 | Loop Period (ms): 15.49 | Sim Time (ms): 13.06
Actual FPS: 0.00 | Loop Period (ms): 15.57 | Sim Time (ms): 12.37
Actual FPS: 0.00 | Loop Period (ms): 21.08 | Sim Time (ms): 15.25
Actual FPS: 0.00 | Loop Period (ms): 15.07 | Sim Time (ms): 12.50
Actual FPS: 0.00 | Loop Period (ms): 21.22 | Sim Time (ms): 15.23
Actual FPS: 0.00 | Loop Period (ms): 27.57 | Sim Time (ms): 21.52
Actual FPS: 0.00 | Loop Period (ms): 13.37 | Sim Time (ms): 11.32
Actual FPS: 24.63 | Loop Period (ms): 18.64 | Sim Time (ms): 16.50
Actual FPS: 24.63 | Loop Period (ms): 16.32 | Sim Time (ms): 14.43
Actual FPS: 24.63 | Loop Period (ms): 19.88 | Sim Time (ms): 14.29
Actual FPS: 24.63 | Loop Period (ms): 19.52 | Sim Time (ms): 17.01
Actual FPS: 24.63 | Loop Period (ms): 35.55 | Sim Time (ms): 32.44
Actua

SystemExit: 

/home/alex/code/muscular_hydrostat/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# End of Demo
The rest below are various debugging things that I need to clean up

In [168]:
# Vectorize face constraints - setup
faces = arm.faces
# faces = faces[:2]

faces = [
    [0, 1, 2, 3],
    [0, 4, 5, 6, 7]
]

flat_faces = []
face_indices = []
for face_index, face in enumerate(faces):
    for vertex in face:
        face_indices.append(face_index)
        flat_faces.append(vertex)

flat_faces = np.array(flat_faces)
face_indices = np.array(face_indices)
_, change_indices, change_counts = np.unique(face_indices, return_index=True, return_counts=True)
change_indices = change_indices + change_counts

# end preprocess

positions = arm.positions
# velocities = arm.velocities
velocities = np.random.random(arm.positions.shape)
points = positions[flat_faces]
dpointsdt = velocities[flat_faces]

cum_pos = np.cumsum(points, axis=0)
pos_sums = np.diff(np.vstack((np.zeros_like(cum_pos[0]), cum_pos[change_indices-1])), axis=0)
centroids = pos_sums/change_counts[:,None]
centered_points = points - centroids[face_indices]

cum_vel = np.cumsum(dpointsdt, axis=0)
vel_sums = np.diff(np.vstack((np.zeros_like(cum_vel[0]), cum_vel[change_indices-1])), axis=0)
dcentroidsdt = vel_sums/change_counts[:,None]
centered_velocities = dpointsdt - dcentroidsdt[face_indices]



In [ ]:
# Vectorize face constraints - covariances
dofs = change_counts - 1

block_points = np.zeros((len(centered_points), len(faces)*3))
col_indices = np.repeat(np.arange(len(faces)*3).reshape(-1,3), change_counts, axis=0)
row_indices = np.repeat(np.arange(len(centered_points)).reshape(-1,1), 3, axis=1)
block_points[row_indices, col_indices] = centered_points
covs = block_points.T @ block_points

row_extract = np.repeat(np.arange(len(covs)).reshape(-1,3), 3, axis=0)
col_extract = np.repeat(np.arange(len(covs)).reshape(-1,1), 3, axis=1)
covs = covs[row_extract, col_extract].reshape(len(faces), 3, 3)

# dCdt
block_vels = np.zeros_like(block_points)
block_vels[row_indices, col_indices] = centered_velocities
dcdt_single = block_vels.T @ block_points
dof_map = np.repeat(np.arange(len(faces)), 3)
dcdt = (dcdt_single + dcdt_single.T) / dofs[dof_map]
dcdt = dcdt[row_extract, col_extract].reshape(len(faces), 3, 3)

# dCdP



In [ ]:
import environment
import obstacle
import polytopes
import numpy as np
import importlib

importlib.reload(environment)
importlib.reload(obstacle)

obst = obstacle.ConvexObstacle3D(
    vertices = (polytopes.Cube.points + np.array([-2, 0, 2])*0),
    edges = polytopes.Cube.edges,
    faces = polytopes.Cube.faces
)

env = environment.Environment3D([obst], np.array([[-.1, -.1, 0.5, 1]]))
# obst_mask = obst.calc_many_intersections(env.grid.reshape(-1, 3)).reshape(100, 100, 100)
# print(np.max(obst_mask +0))

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(projection="3d")
concentration = env.concentration.reshape(-1)
conc_mask = (concentration <= .1) * (concentration > 0.01)
ax.scatter(*env.grid_points[conc_mask,:].T, marker=".")
conc_mask = (concentration == 0)
ax.scatter(*env.grid_points[conc_mask,:].T, marker=".")
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_zlim(0, 10)

In [ ]:
import timeit

arm.calc_next_states(dt = 1/60)
arm.calc_next_states(dt = 1/60)
arm.calc_next_states(dt = 1/60)

positions = arm.positions
velocities = arm.velocities
face_indices = arm.faces[-1]

t = timeit.Timer(lambda: np.diff(arm.positions[arm.edges], axis=1).reshape(-1,3))
print(t.timeit(1000))
print(np.diff(arm.positions[arm.edges], axis=1).shape)
def difference():
    points = arm.positions[arm.edges]
    return points[:,0] - points[:,1]
t = timeit.Timer(lambda: difference())
print(t.timeit(1000))
print(difference().shape)

# points = positions[face_indices]
# dpointsdt = velocities[face_indices]

# centroid = np.average(points, axis=0)
# dcentroiddt = np.average(dpointsdt, axis=0)
# centered_points = points - centroid
# centered_velocities = dpointsdt - dcentroiddt

# dcdt_single = np.einsum("ij,ik->jk", centered_velocities, centered_points)
# # dcdt_single = centered_velocities.T @ centered_points

# t = timeit.Timer(lambda: np.einsum("ij,ik->jk", centered_velocities, centered_points))
# print(t.timeit(1000))

# t = timeit.Timer(lambda: centered_velocities.T @ centered_points)
# print(t.timeit(1000))
# t = timeit.Timer(lambda: hydrostat_arm_3d.calc_face_constraints(positions, velocities, face_indices))
# print(t.timeit(10000)/10)

In [ ]:
# Scent Propogation Demo
import numpy as np
import scipy.signal as sig
diffusion_coef = 1

dx = 0.1
x = np.arange(-5, 5, dx)
y = np.arange(-5, 5, dx)
X, Y = np.meshgrid(x, y)

dt = dx**2/(4*diffusion_coef) * 0.99
final_time = 20
t = np.arange(0, final_time, dt)

concentration = np.zeros((len(t), len(x), len(x)))
concentration[0, 50, 80] = 1

convolve_pattern = np.array([
    [0, 1, 0],
    [1, 0, 1],
    [0, 1, 0]
])
from scipy import ndimage as nd

obst_mask = np.zeros_like(concentration[0])
# obst_mask[np.arange(40,60), 55] = 1
# obst_mask[np.arange(40,60), 45] = 1
# obst_mask[60, np.arange(45,55)] = 1

obst_mask[np.arange(40, 60), np.arange(45, 65)] = 1
adj_obst = sig.convolve2d(obst_mask, convolve_pattern, mode="same", boundary="fill")

for idx, frame in enumerate(concentration):
    if idx == 0: continue
    concentration[idx] = concentration[idx-1] + dt*diffusion_coef*nd.laplace(concentration[idx-1], mode="constant")/dx**2
    concentration[idx, 50, 50] = 1
    concentration[idx] += dt*diffusion_coef/dx**2 * (concentration[idx-1] * adj_obst)
    concentration[idx] = concentration[idx] * (1-obst_mask)

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.animation as anim
fig, ax = plt.subplots()
im = ax.imshow(concentration[-1])

print(np.sum(concentration[-1]))

# def update(frame):
#     im.set_data(concentration[frame])
#     im.set_clim(vmin=np.min(concentration[frame]), vmax=np.max(concentration[frame]))

# ani = anim.FuncAnimation(fig, update, frames=len(concentration)-1, interval=1/60)


In [ ]:
# Double dot
import numpy as np

T1 = np.arange(2*3*4*5).reshape(2, 3, 4, 5)
T2 = np.arange(3*4*5*6).reshape(3, 4, 5, 6)
overlap = 3

T1 = np.arange(2*3*4*5).reshape(2, 3, 4, 5)
T2 = np.arange(4*5*6*7).reshape(4, 5, 6, 7)
overlap = 2

T1 = np.arange(2*3*4*5).reshape(2, 3, 4, 5)
lambdas = np.arange(2)
print(np.tensordot(lambdas, T1, 1).shape)

# a = T1.reshape(T1.shape[:-overlap] + (-1,))
# b = T2.reshape((-1,) + T2.shape[overlap:]).swapaxes(0, -2)
# print(a.shape)
# print(b.shape)
# print(a@b)
# t = timeit.Timer(lambda: (T1.reshape(T1.shape[:-overlap]+(-1,)) @ T2.reshape((-1,) + T2.shape[overlap:]).swapaxes(0, -2)).reshape(T1.shape[:-overlap] + T2.shape[overlap:]))
# print(t.timeit(1000))